In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract results
clip_scores = [5.95, 78.37, 0.14, 0.01, 1.21, 0.02, 0.02, 0.00, 0.63, 13.62]
clips_scores = [76.11, 16.04, 0.01, 0.00, 1.57, 0.00, 0.00, 0.03, 0.77, 5.46]
labels = [f"Caption {i+1}" for i in range(10)]

# Create plot
x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
rects1 = ax.bar(x - width/2, clip_scores, width, label='CLIP')
rects2 = ax.bar(x + width/2, clips_scores, width, label='CLIPS')

# Add labels and legend
ax.set_xlabel('Captions')
ax.set_ylabel('Similarity Score (%)')
ax.set_title('Comparison of CLIP vs CLIPS Similarity Scores')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.tight_layout()
plt.show()

In [1]:
!export CUDA_VISIBLE_DEVICES=1

In [4]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

ERROR: The install method you used for conda--probably either `pip install conda`
or `easy_install conda`--is not compatible with using conda as an application.
If your intention is to install conda as a standalone application, currently
supported install methods include the Anaconda installer and the miniconda
installer.  You can download the miniconda installer from
https://conda.io/miniconda.html.

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-z0h9jzf9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-z0h9jzf9
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369514 sha256=3e4805154f61a7051a81179e6d0303150c85618a93c485092b829a3ef7c14271
  Stored in directory: /tmp/pip-ephem-wheel-cache-vid0yqou/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5

In [ ]:
# Install required dependencies
import torch
import clip
from PIL import Image
import requests
from io import BytesIO
import numpy as np


image = Image.open("sample_image.jpg")


device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model, preprocess = clip.load("ViT-B/32", device=device)


image_input = preprocess(image).unsqueeze(0).to(device)


captions = [
    "A man holding a large dog",
    "A gray Great Dane being held by its owner",
    "A person with a small puppy",
    "A dog sitting on a couch",
    "A man in formal attire with a pet",
    "A woman holding a cat",
    "A gray horse in a stable",
    "A person standing next to a bookshelf",
    "A large dog with its owner in a living room",
    "A man in a white shirt holding a gray dog"
]


text_inputs = torch.cat([clip.tokenize(c) for c in captions]).to(device)


with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)
    

    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    

    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    
    
    print("\nCLIP Similarity Scores:")
    for i, caption in enumerate(captions):
        print(f"{caption}: {similarity[0][i].item():.2%}")


CLIP Similarity Scores:
A man holding a large dog: 5.95%
A gray Great Dane being held by its owner: 78.37%
A person with a small puppy: 0.14%
A dog sitting on a couch: 0.01%
A man in formal attire with a pet: 1.21%
A woman holding a cat: 0.02%
A gray horse in a stable: 0.02%
A person standing next to a bookshelf: 0.00%
A large dog with its owner in a living room: 0.63%
A man in a white shirt holding a gray dog: 13.62%


In [10]:
!pip3 install -r requirements.txt
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.2 MB/s eta 0:00:00a 0:00:01


In [ ]:

import torch
import torch.nn.functional as F
from PIL import Image
from open_clip import create_model_from_pretrained, get_tokenizer


model, preprocess = create_model_from_pretrained('hf-hub:UCSC-VLAA/ViT-L-14-CLIPS-Recap-DataComp-1B')
tokenizer = get_tokenizer('hf-hub:UCSC-VLAA/ViT-L-14-CLIPS-Recap-DataComp-1B')


image = Image.open("sample_image.jpg")
image_input = preprocess(image).unsqueeze(0)


captions = [
    "A man holding a large dog",
    "A gray Great Dane being held by its owner",
    "A person with a small puppy",
    "A dog sitting on a couch",
    "A man in formal attire with a pet",
    "A woman holding a cat",
    "A gray horse in a stable",
    "A person standing next to a bookshelf",
    "A large dog with its owner in a living room",
    "A man in a white shirt holding a gray dog"
]


with torch.no_grad(), torch.cuda.amp.autocast() if torch.cuda.is_available() else torch.no_grad():
    
    text_tokens = tokenizer(captions, context_length=model.context_length)
    
    
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_tokens)
    
    
    image_features = F.normalize(image_features, dim=-1)
    text_features = F.normalize(text_features, dim=-1)
    
    
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    
    
    print("\nCLIPS Similarity Scores:")
    for i, caption in enumerate(captions):
        print(f"{caption}: {similarity[0][i].item():.2%}")

/tmp/ipykernel_3398974/2893933456.py:30: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast() if torch.cuda.is_available() else torch.no_grad():



CLIPS Similarity Scores:
A man holding a large dog: 76.11%
A gray Great Dane being held by its owner: 16.04%
A person with a small puppy: 0.01%
A dog sitting on a couch: 0.00%
A man in formal attire with a pet: 1.57%
A woman holding a cat: 0.00%
A gray horse in a stable: 0.00%
A person standing next to a bookshelf: 0.03%
A large dog with its owner in a living room: 0.77%
A man in a white shirt holding a gray dog: 5.46%


# Analysis of CLIP vs. CLIPS Results

Looking at the similarity scores from both models, there are several interesting patterns and differences worth noting:

## Key Observations

1. **Different Top Predictions:**
   - CLIP strongly favors "A gray Great Dane being held by its owner" (78.37%)
   - CLIPS strongly favors "A man holding a large dog" (76.11%)

2. **Specificity vs. Generality:**
   - CLIP gives higher scores to more specific descriptions (breed identification)
   - CLIPS gives higher scores to more general but accurate descriptions

3. **Secondary Preference:**
   - CLIP's second choice is "A man in a white shirt holding a gray dog" (13.62%)
   - CLIPS's second choice is "A gray Great Dane being held by its owner" (16.04%)

4. **Similar Rejections:**
   - Both models correctly assign very low probabilities to obviously incorrect descriptions like "A woman holding a cat" and "A gray horse in a stable"

5. **Background Element Recognition:**
   - Both models assign low probability to "A person standing next to a bookshelf" despite the bookshelf being visible, suggesting they prioritize the main subjects

## Model Behavior Analysis

**CLIP** appears to be more confident in specific visual details like the breed identification (Great Dane) and color attributes. This suggests CLIP may be better at fine-grained visual categorization when trained on web-crawled pairs that often contain specific nomenclature.

**CLIPS** seems to prioritize the overall scene description and relationship between subjects ("man holding large dog") over specific breed identification. This could reflect its training on synthetic captions that might focus more on relationships and actions rather than specific taxonomic labels.

## Practical Implications

These differences highlight how model training approaches affect what visual-textual relationships are emphasized:

1. **Use Case Considerations:**
   - CLIP might be preferable for applications requiring fine-grained categorization or specific attribute recognition
   - CLIPS might be better for applications focused on understanding relationships and actions between subjects

2. **Caption Style Preferences:**
   - CLIP appears to favor descriptive, taxonomically precise captions
   - CLIPS appears to favor action-oriented, relationship-focused captions

3. **Error Patterns:**
   - Both models effectively reject completely incorrect descriptions
   - Both models assign low probabilities to background elements that are present but not the main focus

This comparison demonstrates that while both models perform the same fundamental task (visual-language alignment), their different architectures and training approaches result in notably different prioritization of visual information. CLIP's emphasis on specific visual features versus CLIPS's emphasis on relationships and actions highlights the importance of model selection based on specific application requirements.
